In [38]:
import os
import numpy
import pandas
import xgboost
from xgboost import XGBRegressor
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import mlflow
from mlflow import MlflowClient
from sklearn.model_selection import cross_val_score
from datetime import datetime

In [6]:
df = pandas.read_csv("/home/kirill/MLOps/OTUSMlOps/data/Data.csv",sep=";")
df

,num,param1,param2,param3,param4,param5,param6,param7,param8,param9,...,param27,param28,param29,param30,param31,param32,param33,param34,param35,param36
0,1,value1,value1,value1,0.064516,0,0,0.622222,0.168421,0.722159,...,0.133333,0.039245,0.075758,0.100000,0.0,0.000010,0.187861,0.202370,0.044586,0.0
1,2,value1,value1,value1,0.096774,0,0,0.344444,0.168421,0.744886,...,0.133333,0.040000,0.075758,0.100000,0.0,0.000009,0.237444,0.196209,0.044586,0.0
2,3,value1,value1,value1,0.129032,0,0,0.316667,0.157895,0.750994,...,0.133333,0.042264,0.075758,0.111111,0.0,0.000010,0.170763,0.193365,0.044586,0.0
3,4,value1,value2,value1,0.161290,0,0,0.322222,0.147368,0.731676,...,0.133333,0.067170,0.106061,0.100000,0.0,0.000007,0.193845,0.242654,0.044586,0.0
4,5,value1,value3,value2,0.064516,0,1,0.394444,0.178947,0.706534,...,0.133333,0.023396,0.060606,0.122222,0.0,0.000005,0.127805,0.196209,0.044586,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18624,18625,value2,value4,value1,0.483871,1,0,0.311111,0.147368,0.690909,...,0.133333,0.034717,0.060606,0.077778,0.0,0.000010,0.140628,0.211374,0.044586,0.0
18625,18626,value2,value4,value1,0.516129,1,0,0.333333,0.149474,0.686648,...,0.133333,0.038491,0.090909,0.111111,0.0,0.000004,0.189998,0.242654,0.044586,0.0
18626,18627,value2,value4,value1,0.548387,1,0,0.611111,0.136842,0.709233,...,0.133333,0.033208,0.075758,0.088889,0.0,0.000007,0.215644,0.240284,0.044586,0.0
18627,18628,value2,value4,value1,0.580645,1,0,0.333333,0.136842,0.716903,...,0.133333,0.039245,0.090909,0.077778,0.0,0.000030,0.200684,0.195261,0.044586,0.0


In [22]:
df['param1'] = df.param1.astype('category')
df['param2'] = df['param2'].astype('category')
df['param3'] = df['param3'].astype('category')
df['param4'] = df['param4'].astype('category')
df.dtypes


num           int64
param1     category
param2     category
param3     category
param4     category
param5        int64
param6        int64
param7      float64
param8      float64
param9      float64
param10     float64
param11     float64
param12     float64
param13     float64
param14     float64
param15     float64
param16     float64
param17     float64
param18     float64
param19     float64
param20     float64
param21     float64
param22     float64
param23     float64
param24     float64
param25     float64
param26     float64
param27     float64
param28     float64
param29     float64
param30     float64
param31     float64
param32     float64
param33     float64
param34     float64
param35     float64
param36     float64
dtype: object

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['param9']), df['param9'], test_size=.2)
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8,enable_categorical=True)
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [34]:
config = {'SOURCE_BUCKET': 'bucket-mlops-project', 
'MLFLOW_URL': 'http://192.168.49.2:30499',
'MODEL_NAME': 'ESPC.model',
'EXPIRIMENT_NAME': 'ESPC_analysis',
'MLFLOW_S3_ENDPOINT_URL': "https://storage.yandexcloud.net",
'FILE_NAME_FOR_REFFIT': "retraining_data.csv",
'FILE_NAME_FOR_TRAIN': "train_data.csv",
'FILE_NAME_FOR_HOLDOUT': "ab_test_data.csv"

}

In [35]:
mlflow.set_tracking_uri(config["MLFLOW_URL"])
client = MlflowClient()
experiment = client.get_experiment_by_name(config["EXPIRIMENT_NAME"])

if experiment is None:
    mlflow.set_experiment(config["EXPIRIMENT_NAME"])
    experiment = client.get_experiment_by_name(config["EXPIRIMENT_NAME"])

experiment_id = experiment.experiment_id

2024/11/29 02:25:23 INFO mlflow.tracking.fluent: Experiment with name 'ESPC_analysis' does not exist. Creating a new experiment.


In [ ]:
import sklearn.metrics


with mlflow.start_run(run_name= 'Run time: ' + ' ' + str(datetime.now()), experiment_id=experiment_id) as run:

    model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8,enable_categorical=True)
    model.fit(X_train, y_train)
      
    mlflow.sklearn.log_model(
            sk_model=model, artifact_path=config["MODEL_NAME"]
        )
    
      
    y_forecast_train = model.predict(X_train)
    train_metric = sklearn.metrics.mean_absolute_percentage_error(y_train, y_forecast_train)
    mlflow.log_metrics({'train_MAPE' : train_metric})
    
    y_forecast = model.predict(X_test)
    test_metric = sklearn.metrics.mean_absolute_percentage_error(y_test, y_forecast)
    mlflow.log_metrics({'test_MAPE' : test_metric})


2024/11/29 04:18:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run  at: http://192.168.49.2:30499/#/experiments/1/runs/69da61a4f4494e26a1f03d5aea3b90b0
🧪 View experiment at: http://192.168.49.2:30499/#/experiments/1
